In [4]:
#==================================================================
#                    INSTALLING DEPENDENCIES
#==================================================================

# import tensorflow as tf
# import glob
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# import librosa as lb
# import librosa.display
# import IPython.display as ipd
# import os
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D,Dense, Dropout,  MaxPooling2D, Flatten
# from tensorflow.keras.metrics import Accuracy, Precision
# from tensorflow.keras.utils import to_categorical
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Dropout, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import Accuracy, Precision

import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import librosa as lb
import librosa.display
import IPython.display as ipd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [5]:
#-------------------------------------------------------------------
#                       Feature Selection
#-------------------------------------------------------------------
# ipd.Audio(audio_files[0])


#using glob and librosa
audio_files = glob.glob("C:\\Datasets\\Data\\genres_original\\*\\*.wav")
SAMPLES_PER_TRACK = 22050 * 30 



# def extract_mel_spectrogram(file_path, n_mels = 128):
#     y, sr = librosa.load(file_path, duration=30)
#     S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
#     S_DB = librosa.power_to_db(S, ref=np.max)
#     return S_DB


def extract_mel_spectrogram(file_path, n_mels=128, duration=30):
    """Extract mel spectrogram with better error handling"""
    try:
        # Try multiple loading methods
        y, sr = librosa.load(file_path, duration=duration, sr=22050)
        
        # Check if audio loaded successfully
        if len(y) == 0:
            print(f"Warning: Empty audio file {file_path}")
            return None
            
        # Generate mel spectrogram
        S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
        S_DB = librosa.power_to_db(S, ref=np.max)
        
        return S_DB
        
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None



In [49]:
#--------------------------------------------------------------------
#                        DATA PREPROCESSING
#--------------------------------------------------------------------


#Below is just normal code for extracting features from DataSet doesnt really help much 
#Kept it as a memorial 
# data = []
# genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
# for genre in genres:
#     folder = f'C:\Datasets\Data\genres_original\{genre}'
#     for filename in os.listdir(folder):
#         file_path = os.path.join(folder, filename)
#         features = extract_features(file_path)
#         data.append([*features, genre])

# df = pd.DataFrame(data)




<string>:19: SyntaxWarning: invalid escape sequence '\{'
<>:19: SyntaxWarning: invalid escape sequence '\{'
<>:19: SyntaxWarning: invalid escape sequence '\D'
<string>:19: SyntaxWarning: invalid escape sequence '\{'
<>:19: SyntaxWarning: invalid escape sequence '\{'
<>:19: SyntaxWarning: invalid escape sequence '\D'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_2332\1467794334.py:19: SyntaxWarning: invalid escape sequence '\{'
  folder = f'C:\Datasets\Data\genres_original\{genre}'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_2332\1467794334.py:19: SyntaxWarning: invalid escape sequence '\D'
  folder = f'C:\Datasets\Data\genres_original\{genre}'


In [9]:
#================================================================================
#                               DATASET LOADING
#================================================================================


#loading Dataset ig
#Preparing for x:y key value thing
# genres = "blues classical country disco hiphop jazz metal pop reggae rock".split(" ")
# X, y = [], []
# for genre in genres:
#     folder = os.path.join(r"C:\Datasets\Data\genres_original", genre)
#     for filename in os.listdir(folder):
#         if filename.endswith(".wav"):
#             filepath = os.path.join(folder, filename)
#             mel = extract_mel_spectrogram(filepath)
#             if mel.shape[1] >= 660:  # Ensure fixed width
#                 mel = mel[:, :660]   # Shape: (128, 660)
#                 X.append(mel)
#                 y.append(genre)

# X = np.array(X)
# X = X[..., np.newaxis]  # Shape: (samples, 128, 660, 1)

# label_encoder = LabelEncoder()
# y_encoded = label_encoder.fit_transform(y)
# y_cat = to_categorical(y_encoded)


def load_dataset_safe(base_path):
    """Load dataset with error handling and progress tracking"""
    genres = "blues classical country disco hiphop jazz metal pop reggae rock".split()
    X, y = [], []
    skipped_files = []
    
    for genre in genres:
        folder = os.path.join(base_path, genre)
        print(f"Processing {genre}...")
        
        if not os.path.exists(folder):
            print(f"Folder {folder} does not exist!")
            continue
            
        file_count = 0
        for filename in os.listdir(folder):
            if filename.endswith(".wav"):
                filepath = os.path.join(folder, filename)
                
                # Extract mel spectrogram
                mel = extract_mel_spectrogram(filepath)
                
                if mel is not None and mel.shape[1] >= 660:  # Ensure minimum width
                    mel = mel[:, :660]   # Shape: (128, 660)
                    X.append(mel)
                    y.append(genre)
                    file_count += 1
                else:
                    skipped_files.append(filepath)
                    
        print(f"Loaded {file_count} files from {genre}")
    
    if skipped_files:
        print(f"\nSkipped {len(skipped_files)} problematic files:")
        for file in skipped_files[:5]:  # Show first 5
            print(f"  {file}")
        if len(skipped_files) > 5:
            print(f"  ... and {len(skipped_files) - 5} more")
    
    # Convert to numpy arrays
    X = np.array(X)
    X = X[..., np.newaxis]  # Shape: (samples, 128, 660, 1)
    
    # Encode labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    y_cat = to_categorical(y_encoded)
    
    print(f"\nDataset loaded successfully!")
    print(f"Total samples: {len(X)}")
    print(f"Input shape: {X.shape}")
    print(f"Output shape: {y_cat.shape}")
    
    return X, y_cat, label_encoder




In [12]:
#=============================================================================
#                               CNN MODEL
#=============================================================================



# #Previous Model (Sucks ass honestly) 
# # Uses x:y system 
# # music_model = Sequential([
# #     Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
# #     Dropout(0.3),
# #     Dense(128, activation='relu'),
# #     Dropout(0.3),
# #     Dense(64, activation='relu'),
# #     Dense(10, activation='softmax')
# # ])

# # music_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',metrics = ['accuracy', 'precision'] )

# # music_model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test))

# #Strings of feature 
# # x = df.iloc[:, :-1]  
# # y = df.iloc[:, -1]   

# #This thing encodes above stuff
# label_encoder = LabelEncoder()
# y = label_encoder.fit_transform(y)

# # Normal stuff yk (train test split) --> NO need to explain
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# #This Model specifically USES Spectrogram given in the data to train CNN(Conv2D and MaxPooling2D) to the music and trains itself
# #Much better than previous stuff
# music_model = Sequential([
#     Conv2D(32, (3, 3), activation='relu', input_shape=(128, 660, 1)),
#     MaxPooling2D((2, 2)),
#     Dropout(0.3),

#     Conv2D(64, (3, 3), activation='relu'),
#     MaxPooling2D((2, 2)),
#     Dropout(0.3),

#     Conv2D(128, (3, 3), activation='relu'),
#     MaxPooling2D((2, 2)),
#     Dropout(0.3),

#     Flatten(),
#     Dense(256, activation='relu'),
#     Dropout(0.4),
#     Dense(10, activation='softmax')
# ])

# music_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'precision'])

# music_model.summary()
# # Train
# music_model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test))



# Load your dataset
base_path = r"C:\Datasets\Data\genres_original"
X, y, label_encoder = load_dataset_safe(base_path)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nTraining set: {X_train.shape}")
print(f"Test set: {X_test.shape}")

# Build the model
music_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 660, 1)),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])



c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 658, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 329, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 63, 329, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 327, 64)    │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 163, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 30, 163, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 161, 128)   │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 80, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 14, 80, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 143360)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │    36,700,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 36,795,658 (140.36 MB)

 Trainable params: 36,795,658 (140.36 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30


ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(None,), output.shape=(None, 10)

In [66]:
#=================================================================
#                      COMPILATION AND TRAINING
#=================================================================

music_model.compile(
    optimizer='adam', 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

print("\nModel Summary:")
music_model.summary()

# Train the model
print("\nStarting training...")
history = music_model.fit(
    X_train, y_train, 
    epochs=30, 
    batch_size=32,
    validation_data=(X_test, y_test),
    verbose=1
)

print("\nTraining completed!")
print(f"Genre classes: {label_encoder.classes_}")

In [67]:
print(genre_names)

['blues' 'classical' 'country' 'disco' 'hiphop' 'jazz' 'metal' 'pop'
 'reggae' 'rock']


In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Dropout, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import Accuracy, Precision

import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import librosa as lb
import librosa.display
import IPython.display as ipd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

def extract_mel_spectrogram(file_path, n_mels=128, duration=30):
    """Extract mel spectrogram with better error handling"""
    try:
        # Try multiple loading methods
        y, sr = librosa.load(file_path, duration=duration, sr=22050)
        
        # Check if audio loaded successfully
        if len(y) == 0:
            print(f"Warning: Empty audio file {file_path}")
            return None
            
        # Generate mel spectrogram
        S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
        S_DB = librosa.power_to_db(S, ref=np.max)
        
        return S_DB
        
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

def load_dataset_safe(base_path):
    """Load dataset with error handling and progress tracking"""
    genres = "blues classical country disco hiphop jazz metal pop reggae rock".split()
    X, y = [], []
    skipped_files = []
    
    for genre in genres:
        folder = os.path.join(base_path, genre)
        print(f"Processing {genre}...")
        
        if not os.path.exists(folder):
            print(f"Folder {folder} does not exist!")
            continue
            
        file_count = 0
        for filename in os.listdir(folder):
            if filename.endswith(".wav"):
                filepath = os.path.join(folder, filename)
                
                # Extract mel spectrogram
                mel = extract_mel_spectrogram(filepath)
                
                if mel is not None and mel.shape[1] >= 660:  # Ensure minimum width
                    mel = mel[:, :660]   # Shape: (128, 660)
                    X.append(mel)
                    y.append(genre)
                    file_count += 1
                else:
                    skipped_files.append(filepath)
                    
        print(f"Loaded {file_count} files from {genre}")
    
    if skipped_files:
        print(f"\nSkipped {len(skipped_files)} problematic files:")
        for file in skipped_files[:5]:  # Show first 5
            print(f"  {file}")
        if len(skipped_files) > 5:
            print(f"  ... and {len(skipped_files) - 5} more")
    
    # Convert to numpy arrays
    X = np.array(X)
    X = X[..., np.newaxis]  # Shape: (samples, 128, 660, 1)
    
    # Encode labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    y_cat = to_categorical(y_encoded)
    
    print(f"\nDataset loaded successfully!")
    print(f"Total samples: {len(X)}")
    print(f"Input shape: {X.shape}")
    print(f"Output shape: {y_cat.shape}")
    
    return X, y_cat, label_encoder

# Load your dataset
base_path = r"C:\Datasets\Data\genres_original"
X, y, label_encoder = load_dataset_safe(base_path)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nTraining set: {X_train.shape}")
print(f"Test set: {X_test.shape}")

# Build the model
music_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 660, 1)),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

music_model.compile(
    optimizer='adam', 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

print("\nModel Summary:")
music_model.summary()

# Train the model
print("\nStarting training...")
history = music_model.fit(
    X_train, y_train, 
    epochs=30, 
    batch_size=32,
    validation_data=(X_test, y_test),
    verbose=1
)

print("\nTraining completed!")
print(f"Genre classes: {label_encoder.classes_}")

Processing blues...
Loaded 100 files from blues
Processing classical...
Loaded 100 files from classical
Processing country...
Loaded 100 files from country
Processing disco...
Loaded 100 files from disco
Processing hiphop...
Loaded 100 files from hiphop
Processing jazz...
Loaded 99 files from jazz
Processing metal...
Loaded 100 files from metal
Processing pop...
Loaded 100 files from pop
Processing reggae...
Loaded 100 files from reggae
Processing rock...
Loaded 100 files from rock

Dataset loaded successfully!
Total samples: 999
Input shape: (999, 128, 660, 1)
Output shape: (999, 10)

Training set: (799, 128, 660, 1)
Test set: (200, 128, 660, 1)

Model Summary:


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 126, 658, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 329, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 63, 329, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 327, 64)    │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 163, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 30, 163, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 161, 128)   │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 80, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 14, 80, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 143360)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │    36,700,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 36,795,658 (140.36 MB)

 Trainable params: 36,795,658 (140.36 MB)

 Non-trainable params: 0 (0.00 B)


Starting training...
Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 54s 2s/step - accuracy: 0.1100 - loss: 321.1418 - val_accuracy: 0.1000 - val_loss: 2.3028
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 45s 2s/step - accuracy: 0.1095 - loss: 2.3012 - val_accuracy: 0.1100 - val_loss: 2.3031
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 45s 2s/step - accuracy: 0.0845 - loss: 2.3050 - val_accuracy: 0.0900 - val_loss: 2.3035
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.1178 - loss: 2.3036 - val_accuracy: 0.0900 - val_loss: 2.3040
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step - accuracy: 0.1044 - loss: 2.3022 - val_accuracy: 0.0750 - val_loss: 2.3046
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 45s 2s/step - accuracy: 0.1102 - loss: 2.3023 - val_accuracy: 0.0600 - val_loss: 2.3052
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step - accuracy: 0.0916 - loss: 2.3002 - val_accuracy: 0.0600 - val_loss: 2.3056
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 48s 2s/step - accuracy: 0.0947 - loss: 2.3045 - val_accu

KeyboardInterrupt: 